In [14]:
import os
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

In [15]:
# 현재경로 확인
os.getcwd()

'C:\\Users\\hyose\\Desktop\\데이터 공부\\ML\\4주차 앙상블\\2. 실습\\실습 코드'

In [16]:
# 데이터 불러오기
data = pd.read_csv("./데이터/kc_house_data.csv") 
data.head() # 데이터 확인

,id,date,price,bedrooms,bathrooms,floors,waterfront,condition,grade,yr_built,yr_renovated,zipcode,lat,long
0,7129300520,20141013T000000,221900.0,3,1.00,1.0,0,3,7,1955,0,98178,47.5112,-122.257
1,6414100192,20141209T000000,538000.0,3,2.25,2.0,0,3,7,1951,1991,98125,47.7210,-122.319
2,5631500400,20150225T000000,180000.0,2,1.00,1.0,0,3,6,1933,0,98028,47.7379,-122.233
3,2487200875,20141209T000000,604000.0,4,3.00,1.0,0,5,7,1965,0,98136,47.5208,-122.393
4,1954400510,20150218T000000,510000.0,3,2.00,1.0,0,3,8,1987,0,98074,47.6168,-122.045


In [17]:
'''
id: 집 고유아이디
date: 집이 팔린 날짜 
price: 집 가격 (타겟변수)
bedrooms: 주택 당 침실 개수
bathrooms: 주택 당 화장실 개수
floors: 전체 층 개수
waterfront: 해변이 보이는지 (0, 1)
condition: 집 청소상태 (1~5)
grade: King County grading system 으로 인한 평점 (1~13)
yr_built: 집이 지어진 년도
yr_renovated: 집이 리모델링 된 년도
zipcode: 우편번호
lat: 위도
long: 경도
'''

'\nid: 집 고유아이디\ndate: 집이 팔린 날짜 \nprice: 집 가격 (타겟변수)\nbedrooms: 주택 당 침실 개수\nbathrooms: 주택 당 화장실 개수\nfloors: 전체 층 개수\nwaterfront: 해변이 보이는지 (0, 1)\ncondition: 집 청소상태 (1~5)\ngrade: King County grading system 으로 인한 평점 (1~13)\nyr_built: 집이 지어진 년도\nyr_renovated: 집이 리모델링 된 년도\nzipcode: 우편번호\nlat: 위도\nlong: 경도\n'

In [18]:
nCar = data.shape[0] # 데이터 개수
nVar = data.shape[1] # 변수 개수
print('nCar: %d' % nCar, 'nVar: %d' % nVar )

nCar: 21613 nVar: 14


## 의미가 없다고 판단되는 변수 제거

In [19]:
data = data.drop(['id', 'date', 'zipcode', 'lat', 'long'], axis = 1) # id, date, zipcode, lat, long  제거

## 범주형 변수를 이진형 변수로 변환
- 범주형 변수는 waterfront 컬럼 뿐이며, 이진 분류이기 때문에 0, 1로 표현한다.
- 데이터에서 0, 1로 표현되어 있으므로 과정 생략

## 설명변수와 타겟변수를 분리, 학습데이터와 평가데이터 분리

In [20]:
feature_columns = list(data.columns.difference(['price'])) # Price를 제외한 모든 행
X = data[feature_columns]
y = data['price']
train_x, test_x, train_y, test_y = train_test_split(X, y, test_size = 0.3, random_state = 42) # 학습데이터와 평가데이터의 비율을 7:3
print(train_x.shape, test_x.shape, train_y.shape, test_y.shape) # 데이터 개수 확인

(15129, 8) (6484, 8) (15129,) (6484,)


## 학습 데이터를 선형 회귀 모형에 적합 후 평가 데이터로 검증 (Scikit_learn)

In [21]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score
from math import sqrt

regression_model = LinearRegression() # 선형 회귀 모형
linear_model1 = regression_model.fit(train_x, train_y) # 학습 데이터를 선형 회귀 모형에 적합
predict1 = linear_model1.predict(test_x) # 학습된 선형 회귀 모형으로 평가 데이터 예측
print("RMSE: {}".format(sqrt(mean_squared_error(predict1, test_y)))) # RMSE 결과

RMSE: 239804.29670858147


BaggingRegressor에는 model parameter가 존재한다. 즉, 학습모델을 parameter로 넣어야 한다는 뜻이다. 이는 DecisionTree가 될 수도 있고, SVM 등등 다른 모델이 될 수도 있다. 이제 이 regression_model을 활용하여 bagging함수를 적용시켜보자. 

## Bagging 을 이용하여 선형 회귀 모형에 적합 후 평가 (Sampling 10번)

In [22]:
# bagging 진행
from sklearn.ensemble import BaggingRegressor
bagging_model = BaggingRegressor(base_estimator = regression_model, # 선형회귀모형
                                 n_estimators = 5, # 5번 샘플링
                                 verbose = 1) # 학습 과정 표시
linear_model2 = bagging_model.fit(train_x, train_y) # 학습 진행
predict2 = linear_model2.predict(test_x) # 학습된 Bagging 선형 회귀 모형으로 평가 데이터 예측
print("RMSE: {}".format(sqrt(mean_squared_error(predict2, test_y)))) # RMSE 결과

RMSE: 240032.1929318346


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s finished


위의 결과는 BaggingRegressor을 활용하여 bagging기법을 적용시킨 RMSE이다. 이 또한 For문으로 구현하는 것에 비해 크게 달라진 것이 없다. 물론 반복복원추출이기에 ensemble기법을 적용시킬 때마다 다른결과가 나오지만 그렇게 큰 차이가 생기진 않는다. 만약 같은 ensemble기법과 학습모델을 선택한다면 parameter를 경험적으로 조정해보는 것이 훨씬 더 많은 성능차이를 야기할 수도 있다.

 그리고 아래 빨간 블럭으로 출력이 된 결과는 verbose=1로 표시했기에 학습과정을 출력한다는 의미이다. 이제 parameter를 조정하는 과정을 진행해보자. 


## 그렇다면 Sampling을 많이 해보자!

In [23]:
bagging_model2 = BaggingRegressor(base_estimator = regression_model, # 선형 회귀모형
                                  n_estimators = 30, # 30번 샘플링
                                  verbose = 1) # 학습 과정 표시
linear_model3 = bagging_model2.fit(train_x, train_y) # 학습 진행
predict3 = linear_model3.predict(test_x) # 학습된 Bagging 선형 회귀 모형으로 평가 데이터 예측
print("RMSE: {}".format(sqrt(mean_squared_error(predict3, test_y)))) # RMSE 결과

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s finished


RMSE: 239853.53997595128


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s finished


이전 학습과 다른점은 샘플링 개수를 늘린 것이다. 5개에서 30개로 늘리면 시간은 그만큼 더 많이 소요된다. 물론 학습결과 또한 좀 더 나아졌다. 하지만 이 차이를 선형회귀모델에서 크게 느끼기는 힘들다. 

 그렇다면, 이제 모델을 DecisionTree로 바꾸었을때, Bagging을 진행해보자.

## 학습 데이터를 의사결정나무모형에 적합 후 평가 데이터로 검증

In [24]:
from sklearn.tree import DecisionTreeRegressor
decision_tree_model = DecisionTreeRegressor() # 의사결정나무 모형
tree_model1 = decision_tree_model.fit(train_x, train_y) # 학습 데이터를 의사결정나무 모형에 적합
predict1 = tree_model1.predict(test_x) # 학습된 의사결정나무 모형으로 평가 데이터 예측
print("RMSE: {}".format(sqrt(mean_squared_error(predict1, test_y)))) # RMSE 결과

RMSE: 289452.6253578015


Decision Tree의 기본적인 error는 위와 같다. DecisionTreeRegressor의 특성상 구간예측을 하는 경향이 어느정도 강하기에 정확한 MSE가 높을 수 있다. 이를 Bagging작업을 통해 오차가 줄어드는지 확인해보자. 



## Bagging 을 이용하여 의사결정나무모형에 적합 후 평가 (Sampling 10번)

In [25]:
bagging_decision_tree_model1 = BaggingRegressor(base_estimator = decision_tree_model, # 의사결정나무 모형
                                                n_estimators = 5, # 5번 샘플링
                                                verbose = 1) # 학습 과정 표시
tree_model2 = bagging_decision_tree_model1.fit(train_x, train_y) # 학습 진행
predict2 = tree_model2.predict(test_x) # 학습된 Bagging 의사결정나무 모형으로 평가 데이터 예측
print("RMSE: {}".format(sqrt(mean_squared_error(predict2, test_y)))) # RMSE 결과

RMSE: 247962.9422705334


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s finished


In [26]:
bagging_decision_tree_model2 = BaggingRegressor(base_estimator = decision_tree_model, # 의사결정나무 모형
                                                n_estimators = 30, # 30번 샘플링
                                                verbose = 1) # 학습 과정 표시
tree_model3 = bagging_decision_tree_model2.fit(train_x, train_y) # 학습 진행
predict3 = tree_model3.predict(test_x) # 학습된 Bagging 의사결정나무 모형으로 평가 데이터 예측
print("RMSE: {}".format(sqrt(mean_squared_error(predict3, test_y)))) # RMSE 결과

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


RMSE: 230645.4439767361


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.4s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s finished


 먼저 DecisionTreeRegressor의 경우 bagging작업을 한 오차가 단순 DecisionTree보다 눈에 띄게 오차를 줄인 것을 확인할 수 있다. 실제로 앙상블 모형들이 Tree기반인 모형이 많은 이유 또한 위의 결과와 같다. 선형회귀의 경우는 Overfitting이 잘 일어나지 않으며, 좋아지는데 한계가 명확하지만, 그렇지 않은 모델일 경우 Ensemble기법은 큰 성능차이를 발생시킬 수 있다. 

 게다가 두 번째처럼 sampling의 개수를 늘리게 되면, 즉 학습을 더 많은 경우로 나누어 시키고 종합시킨다면 error또한 더 낮아지는 것을 확인할 수 있다. 물론 이 error의 감소는 무한하지 않다. 저 sampling 개수도 최적의 n개가 존재할 수 있다. 하지만, 정해진 것은 없기에 경험적으로 찾아야 할 필요가 있다. 